In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 107.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.139
    Uninstalling botocore-1.29.139:
      Successfully uninstalled botocore-1.29.139
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.139
    Uninstalling boto3-1.26.139:
      Successfully uninstalled boto3-1.26.139
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.114 requires botocore==1.29.114, but you have botocore 1.29.140 which is incompatible.
awscli 1.27.114 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://p

In [2]:
import boto3
import sagemaker

account_id        = boto3.client('sts').get_caller_identity().get('Account')
region_name       = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket            = sagemaker_session.default_bucket()
role              = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::928808346782:role/SageMakerMLOpsRole
sagemaker-us-west-2-928808346782


In [3]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatglm')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatglm')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-928808346782/chatglm/assets/model.tar.gz


In [4]:
model_name = None
entry_point = 'chatglm-inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT': '600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [5]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name              = model_name,
    model_data        = model_data,
    entry_point       = entry_point,
    source_dir        = './code',
    role              = role,
    framework_version = framework_version, 
    py_version        = py_version,
    env               = model_environment
)

In [6]:

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name         = None
instance_type         = 'ml.g4dn.2xlarge'
instance_count        = 1

predictor = model.deploy(
    endpoint_name          = endpoint_name,
    instance_type          = instance_type, 
    initial_instance_count = instance_count,
    serializer             = JSONSerializer(),
    deserializer           = JSONDeserializer()
)

----------!

In [23]:
inputs = {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])


你好！请问有什么需要帮助的吗？


In [10]:
inputs = {
    "ask": "类型#上衣\*材质#牛仔布\*颜色#白色\*风格#简约\*图案#刺绣\*衣样式#外套\*衣款式#破洞"

}

response = predictor.predict(inputs)
print(response["answer"])


1. 类型：上衣
* 材质：牛仔布
* 颜色：白色
* 风格：简约
* 图案：刺绣
* 衣样式：无样式
* 外套：无外套
